# Парсер отчетов АО «Открытие Брокер»

In [2]:
import xml.etree.ElementTree as ET
from datetime import datetime as dt
import pandas as pd
from pandas import to_datetime as to_dt
import numpy as np
from os import listdir, makedirs
from os.path import isfile, join, exists
import re

### 1. Считываем отчеты, которые находятся в папке Data:

In [3]:
my_files_list = [join('Data/', f) for f in listdir('Data/') if isfile(join('Data/', f))]
my_xml_data = []

# Считывание отчетов из каталога
for f in my_files_list:
    tree = ET.parse(f)
    root = tree.getroot()
    my_xml_data.append(root)

### 2. Словари для основных оцениваемых параметров

Общие словари для итоговой выборки:

In [4]:
dict_stocks = {'stock_name': [], 'account': [], 'currency': [], 'current_cost': [], 'current_cost_rub': [], 'saldo' : []}
dict_deals  = {'stock_name': [], 'account': [], 'date_oper': [], 'type_oper': [], 'quantity': [], 'price': [], 'currency': [], 'brokerage': [], 'result': []}
dict_flows  = {'stock_name': [], 'account': [], 'date_oper': [], 'type_oper': [], 'result': [], 'currency': []}
dict_avg_price = {'stock_name': [], 'account': [], 'avg_open_price' : []}

### 3. Маппинг тегов из отчетов для приведения к единому словарю
Т.к. форматы отчетов отличаются между, необходима небольшая кастомизация и маппинг:

In [5]:
tags_mapping = {
    'SPB': {
        'current_position': 'briefcase_position',
        'deals': 'closed_deal',
        'flows': 'nontrade_money_operation',
        'stock_name': 'issuername',
        'current_cost': 'clearingprice',
        'current_cost_rub': 'valuationrur',
        'saldo': 'saldo',
        'ticketdate': 'ticketdate',
        'price': 'price',
        'brokerage': 'brokerage',
        'operationdate': 'operationdate',
        'currency': 'pricecurrencycode',
        'stock_name_deal': 'issuername',
        'paymentcurrency': 'paymentcurrency',
        'currency_flows': 'currencycode'
    },
    'MOEX': {
        'current_position': 'spot_assets',
        'deals': 'spot_main_deals_conclusion',
        'flows': 'spot_non_trade_money_operations',
        'stock_name': 'asset_name',
        'current_cost': 'settlement_price',
        'current_cost_rub': 'settlement_fact',
        'saldo': 'closing_position_fact',
        'ticketdate': 'conclusion_date',
        'price': 'price',
        'brokerage': 'broker_commission',
        'operationdate': 'operation_date',
        'currency': 'price_currency',
        'stock_name_deal': 'security_name',
        'paymentcurrency': 'price_currency_code',
        'currency_flows': 'currency_code'
    }
}

### 4. Класс для расчета средней цены открытой позиции:

Основная идея расчета доходности по сделке и средней цены открытой позиции взята отсюда: https://lichgo.github.io/2015/10/29/40-lines-pnl-calculation.html

In [6]:
class PnlSnapshot:
    def __init__(self, ticker, traded_price, traded_quantity):
        self.m_ticker = ticker
        self.m_net_position = 0
        self.m_avg_open_price = 0
        self.m_net_investment = 0
        self.m_realized_pnl = 0
        self.m_unrealized_pnl = 0
        self.m_total_pnl = 0
        self.update_by_tradefeed(traded_price, traded_quantity)

    def update_by_tradefeed(self, traded_price, traded_quantity):
        quantity_with_direction = traded_quantity
        is_still_open = (self.m_net_position * quantity_with_direction) >= 0
        # net investment
        self.m_net_investment = max( self.m_net_investment, abs( self.m_net_position * self.m_avg_open_price  ) )
        # realized pnl
        if not is_still_open:
            self.m_realized_pnl = ( traded_price - self.m_avg_open_price ) * \
                min( 
                    abs(quantity_with_direction), 
                    abs(self.m_net_position) 
                ) * ( abs(self.m_net_position) / self.m_net_position )
        else:
            self.m_realized_pnl = 0


        # total pnl
        self.m_total_pnl = self.m_realized_pnl + self.m_unrealized_pnl
        # avg open price
        if is_still_open:
            self.m_avg_open_price = ( ( self.m_avg_open_price * self.m_net_position ) + 
                ( traded_price * quantity_with_direction ) ) / ( self.m_net_position + quantity_with_direction )
        else:
            # Check if it is close-and-open
            if traded_quantity > abs(self.m_net_position):
                self.m_avg_open_price = traded_price
        # net position
        self.m_net_position += quantity_with_direction

    def update_by_marketdata(self, last_price):
        self.m_unrealized_pnl = ( last_price - self.m_avg_open_price ) * self.m_net_position
        self.m_total_pnl = self.m_realized_pnl + self.m_unrealized_pnl

### 5. Вспомогательные функции

#### Функция account_name необходима для получения имени счета по названию биржи и номер счета
например, если биржа MOEX и номер счета вида XXXXXXi, то это - ИИС, если XXXXXX, то это обычный брокерский счет на бирже MOEX

In [7]:
# Функция получения имени счета по наименованию биржи и клиентского номера
def get_account_name(exchange_name, client_code):

    if exchange_name == 'MOEX':
        if client_code[-1].lower() == 'i':
            acc_name = 'MOEX IIS'
        else:
            acc_name = 'MOEX BROK'
    else:
        acc_name = 'SPB BROK'
        
    return acc_name

#### Функция маппинга тега отчета в зависимости от площадки

In [8]:
def get_allias(exchange_name):
    return(
            tags_mapping[exchange_name]['current_position'],
            tags_mapping[exchange_name]['deals'],
            tags_mapping[exchange_name]['flows'],
            tags_mapping[exchange_name]['stock_name'],
            tags_mapping[exchange_name]['saldo'],
            tags_mapping[exchange_name]['ticketdate'],
            tags_mapping[exchange_name]['price'],
            tags_mapping[exchange_name]['brokerage'],
            tags_mapping[exchange_name]['operationdate'],
            tags_mapping[exchange_name]['currency'],
            tags_mapping[exchange_name]['current_cost'],
            tags_mapping[exchange_name]['current_cost_rub'],
            tags_mapping[exchange_name]['stock_name_deal'],
            tags_mapping[exchange_name]['paymentcurrency'],
            tags_mapping[exchange_name]['currency_flows']
          )

#### Функция заполнения словаря по позициям клиентского портфеля

Одна из главных сложностей - отсутствие унификации в обозначании наименования ценной бумаги в разных частях отчетов, например:
1. В блоке отчета (briefcase_position) имеется уникальный идентфиикатор (isin) акции Microsoft, наименование тикетра (MSFT) и само название эмитента - Microsoft;
2. В блоке по закрытым сделкам (closed_deal) есть аналогичная информация, тут проблем нет;
3. Но в блоке по неторговым операциям (nontrade_money_operation) кроме как в комментарии никакой дополнительной информации нет. Например, "comment="Выплата дохода клиент &lt;777777&gt; дивиденды &lt;MICROSOFT COM-ао&gt;". И кроме как соотнести по названию этот 'MICROSOFT COM-ао' с 'Microsoft' из указанных выше таблиц, вариантов иных нет

In [9]:
def get_briefcase(XMLdata):
   
    # В отчете ФБ СПБ портфель находится под тегом briefcase_position
    briefcase_position = XMLdata.find(current_position)
    if not briefcase_position:
        return

    try:
        for child in briefcase_position:
            stock_name_reduce = child.get(stock_name).upper()
            stock_name_reduce = re.sub('[,\.]|(\s?INC)|(\s+$)|([-\s]?АО)', '', stock_name_reduce)

            dict_stocks['stock_name'].append(stock_name_reduce)
            dict_stocks['account'].append(account_name)
            dict_stocks['currency'].append(child.get(currency))
            dict_stocks['current_cost'].append(float(child.get(current_cost)))
            dict_stocks['current_cost_rub'].append(float(child.get(current_cost_rub)))
            dict_stocks['saldo'].append(float(child.get(saldo)))

    except Exception as e:
        print('get_briefcase --> Oops! It seems we have a BUG!', e)  

#### Функция установки средней цены в Dataframe

In [10]:
def put_avr_price_in_df(account, stock_name_proc, net_position, avg_open_price):
    
    if stock_name_proc in df_stocks['stock_name'].values:
        criterion = df_stocks['stock_name'] == stock_name_proc
        criterion = criterion & (df_stocks['account'] == account)
   
        if not df_stocks.loc[criterion].empty:
    
            if net_position > 0:
                dict_avg_price['account'].append(account)
                dict_avg_price['stock_name'].append(stock_name_proc)
                dict_avg_price['avg_open_price'].append(avg_open_price)

#### Функция получения цекущей цены бумаги

In [11]:
def get_current_cost(stock_name):
    current_cost = 0
    
    if stock_name in df_stocks.index:
        current_cost = df_stocks.loc[stock_name, 'current_cost']
    
    return current_cost

#### Функция заполнения словаря по заключенным сделкам

In [12]:
def sortchildrenby(parent, attr):
    parent[:] = sorted(parent, key=lambda child: child.get(attr))

In [13]:
def get_deals(XMLdata):

    stock_name_proc = ''    
    
    closed_deal = XMLdata.find(deals)
    if not closed_deal:
        return
    # Отчет по SPB имеет иную сортировку - только по дате сделки,
    # тогда как отчеты MOEX: по бумаге, а потом по дате сделки
    # Отсортируем сделки по бумаге:
    if exchange_name == 'SPB':
        sortchildrenby(closed_deal, stock_name_deal)
        for child in closed_deal:
            sortchildrenby(child, stock_name_deal)
    try:        
        for child in closed_deal:
            stock_name_reduce = child.get(stock_name_deal).upper()
            stock_name_reduce = re.sub('[,\.]|(\s?INC)|(\s+$)|([-\s]?АО)', '', stock_name_reduce)

            dict_deals['stock_name'].append(stock_name_reduce)
            dict_deals['account'].append(account_name)
            dict_deals['date_oper'].append(to_dt(child.get(ticketdate)).strftime('%Y-%m-%d'))

            current_cost = get_current_cost(stock_name_reduce)

            # В отчете по SPB один тег на количество - quantity,
            # а на MOEX целых два: buy_qnty и sell_qnty
            if exchange_name == 'MOEX':
                if child.get('buy_qnty'):
                    quantity = float(child.get('buy_qnty'))
                else:
                    quantity = - float(child.get('sell_qnty'))
            else:    
                quantity = float(child.get('quantity'))

            dict_deals['quantity'].append(quantity)    
            dict_deals['price'].append(float(child.get('price')))
            dict_deals['type_oper'].append('deal')
            dict_deals['currency'].append(child.get(payment_currency))

            brok_comm = child.get(brokerage)
            if brok_comm is None: 
                brok_comm = 0
            else:
                brok_comm = float(brok_comm)
            dict_deals['brokerage'].append(float(brok_comm))

            # Доходность по каждой сделке и средняя цена позиции
            if stock_name_proc != stock_name_reduce:

                if stock_name_proc != '':
                    put_avr_price_in_df(account_name, stock_name_proc, \
                                        pnl.m_net_position, pnl.m_avg_open_price)

                    current_cost = get_current_cost(stock_name_proc)
                    pnl.update_by_marketdata(current_cost)
                    if len(dict_deals['result']) > 0: 
                        if exchange_name != 'SPB':
                            dict_deals['result'][-1] = pnl.m_unrealized_pnl * 0.87 -dict_deals['brokerage'][-2]
                        else:
                            dict_deals['result'][-1] = pnl.m_unrealized_pnl - dict_deals['brokerage'][-2]

                stock_name_proc = stock_name_reduce
                pnl = PnlSnapshot(stock_name_proc, float(child.get('price')), quantity)
                dict_deals['result'].append(-1 * brok_comm)
            else:
                pnl.update_by_tradefeed(float(child.get('price')), quantity)

                # Продажа бумаг, фиксация результата
                if quantity < 0:
                    if pnl.m_realized_pnl > 0 and exchange_name != 'SPB':
                        pnl_sum = pnl.m_realized_pnl * 0.87 - brok_comm
                    else:
                        pnl_sum = pnl.m_realized_pnl - brok_comm

                    dict_deals['result'].append(float(pnl_sum))
                else:
                    pnl.update_by_marketdata(current_cost)
                    dict_deals['result'].append(-1 * brok_comm)

        put_avr_price_in_df(account_name, stock_name_proc, \
                            pnl.m_net_position, pnl.m_avg_open_price)

        current_cost = get_current_cost(stock_name_proc)
        pnl.update_by_marketdata(current_cost)
        if len(dict_deals['result']) > 0: 
            if exchange_name != 'SPB':
                dict_deals['result'][-1] = pnl.m_unrealized_pnl * 0.87 -dict_deals['brokerage'][-2]
            else:
                dict_deals['result'][-1] = pnl.m_unrealized_pnl - dict_deals['brokerage'][-2]

    except Exception as e:
        print('get_deals --> Oops! It seems we have a BUG!', e)  

#### Функция извлечения с помощью регулярных выражений информацию об эмитенте.

Варианты могут быть примерно следующими:
1. Выплата дохода клиент &lt;777777&gt; дивиденды &lt;APPLE INC-ао&gt; --> выплата дивидендов из отчета SPB
2. Выплата дохода клиент 777777i (НКД 2 ОФЗ 24019) налог к удержанию 0.00 рублей --> выплата купона из отчета MOEX
3. Выплата дохода клиент 777777 дивиденды ФСК ЕЭС-ао налог к удержанию 76.00 рублей --> выплата дивидендов из отчета MOEX

In [14]:
def get_company_from_str(comment):
    company_name = ''
    
    # Шаблоны для случаев дивиденды/купон
    flows_pattern = [
        '^.+дивиденды\s<(\w+)?.+-ао>$',
        '^.+дивиденды\s(.+)-а.+$',
        '^.+\(НКД\s\d?\s(.+)\).+$',
        '^.+дивидендам\s(.+)-.+$'
    ]
    
    for pattern in flows_pattern:
        match = re.search(pattern, comment)
        if match:
            return match.group(1).upper()
   
    return company_name

#### Функция приведения наименовании компании к единому справочнику
В качестве справочника используется блок - заключенные сделки, т.к. в клиентском портфеле может уже не быть тех бумаг, по которым когда-то платились дивиденды

In [15]:
def get_company_from_briefcase(company_name):
    company_name_full = None
        
    value_from_dic = df_deals[df_deals['stock_name'].str.contains(company_name)]
    company_arr  = value_from_dic['stock_name'].unique()
    
    if len(company_arr) == 1:
        company_name_full = company_arr[0]
       
    return company_name_full

In [16]:
def get_full_name(comment):
    company_name = get_company_from_str(comment)
    company_name_full = get_company_from_briefcase(company_name)
    return company_name_full

#### Функция определения типа неторговой операции: дивиденды, купон или удержание налога

In [17]:
def get_type_oper(comment):
    type_oper = 'div'
    type_oper_pattern = {
        'дивиденды' : 'div',
        'НКД' : 'NKD',
        '^.+налог.+дивидендам.+$' : 'tax'
    }
    
    for key, value in type_oper_pattern.items():
        match = re.search(key, comment)
        if match:
            type_oper = value
            break
   
    return type_oper

#### Функция заполнения словаря по неторговым операциям по счету. 

Технически, самая сложная часть алгоритма, т.к. в исходных данных нет четких признаков операции и однозначной привязки к определенной ценной бумаге, придется использовать регулярные выражения и парсить текст, выдергивая оттуда название.

В отчете по бирже SPB имеется интересный тег - analyticname, который может содержать указание на вид операции (Дивиденды, комиссия, перевод между прощадками и т.д.). В отчетах по MOEX такого тега нет, поэтому будет использовать универсальное решение: парсить весь текст по ключевым словам:

In [18]:
def get_nontrade_operation(XMLdata):
    nontrade_money_operation = XMLdata.find(flows)

    if not nontrade_money_operation:
        return
        
    try:
        for child in nontrade_money_operation:

            comment = child.get('comment')
            type_oper_match = re.search('дивиденды|НКД|^.+налог.+дивидендам.+$', comment) 

            if type_oper_match:

                company_name = get_company_from_str(comment)
                type_oper = get_type_oper(comment)

                dict_flows['stock_name'].append(company_name)
                dict_flows['account'].append(account_name)
                dict_flows['date_oper'].append(to_dt(child.get(operationdate)).strftime('%Y-%m-%d'))
                dict_flows['type_oper'].append(type_oper)
                dict_flows['result'].append(float(child.get('amount')))
                dict_flows['currency'].append(child.get(currency_flows))

    except Exception as e:
        print('get_nontrade_operation --> Oops! It seems we have a BUG!', e)  

### 6. Сбор и обработка информации из отчетов

Основная часть алгоритма - это консолидация информации из нужных блоков отчета:

In [19]:
# Сбор данных из соответствующих частей отчетов
for XMLdata in my_xml_data:
    # Информация о Бирже и счете
    exchange_name = 'SPB' if XMLdata.get('board_list') == 'ФБ СПБ' else 'MOEX' 
    client_code =  XMLdata.get('client_code')
    account_name = get_account_name(exchange_name, client_code)
    
    # Маппинг тегов
    current_position, deals, flows, stock_name, \
    saldo, ticketdate, price, brokerage, \
    operationdate, currency, \
    current_cost, current_cost_rub, \
    stock_name_deal, payment_currency, currency_flows = get_allias(exchange_name)
    
    # Информация о состоянии клиентского портфеля
    get_briefcase(XMLdata)
    df_stocks = pd.DataFrame(dict_stocks)
    df_stocks.set_index("stock_name", drop = False, inplace = True)
       
    # Информация о сделках
    get_deals(XMLdata)
    df_deals = pd.DataFrame(dict_deals)
    df_avg = pd.DataFrame(dict_avg_price)
    
    # Информация о неторговых операциях по счету
    get_nontrade_operation(XMLdata)
    df_flows = pd.DataFrame(dict_flows)

In [204]:
df_stocks_avg = pd.merge(df_stocks, df_avg, how='outer', on=['account', 'stock_name', ])
df_stocks_avg.sample(5)

,account,currency,current_cost,current_cost_rub,saldo,stock_name,avg_open_price
1,MOEX IIS,RUB,51.69500,31017.00,600.0,ММК,36.169643
4,SPB BROK,USD,114.74000,15053.44,2.0,MICROSOFT,75.500000
3,SPB BROK,USD,225.35000,118449.47,8.0,APPLE,148.026250
0,MOEX BROK,RUB,0.16745,8372.50,50000.0,ФСК ЕЭС,0.245986
2,MOEX IIS,RUB,100.48900,290987.20,280.0,ОФЗ 24019,102.115960


In [205]:
df_deals.sample(10)

,account,brokerage,currency,date_oper,price,quantity,result,stock_name,type_oper
5,MOEX BROK,8.37,RUB,2017-02-27,0.20920,-20000.0,-3420.843571,ФСК ЕЭС,deal
30,SPB BROK,0.05,USD,2017-09-20,158.30000,1.0,-0.050000,APPLE,deal
10,MOEX IIS,4.69,RUB,2017-02-07,41.10000,200.0,-4.690000,ММК,deal
24,MOEX IIS,11.69,RUB,2017-12-28,102.53200,20.0,-11.690000,ОФЗ 24019,deal
7,MOEX IIS,7.67,RUB,2017-01-03,33.65500,400.0,-7.670000,ММК,deal
13,MOEX IIS,19.12,RUB,2017-03-28,101.63320,33.0,-19.120000,ОФЗ 24019,deal
26,SPB BROK,0.00,USD,2016-11-30,110.77000,1.0,0.000000,APPLE,deal
3,MOEX BROK,5.09,RUB,2017-02-09,0.25440,10000.0,-5.090000,ФСК ЕЭС,deal
0,MOEX BROK,15.96,RUB,2016-12-01,0.19955,40000.0,-15.960000,ФСК ЕЭС,deal
21,MOEX IIS,8.74,RUB,2017-11-22,102.22000,15.0,-8.740000,ОФЗ 24019,deal


In [206]:
df_flows.sample(10)

,account,currency,date_oper,result,stock_name,type_oper
13,MOEX IIS,RUB,2018-01-10,666.60,ММК,div
28,SPB BROK,USD,2018-08-23,5.26,APPLE,div
4,MOEX BROK,RUB,2018-08-01,-76.00,ФСК ЕЭС,tax
21,SPB BROK,USD,2017-08-23,2.27,APPLE,div
0,MOEX BROK,RUB,2017-08-01,-91.00,ФСК ЕЭС,tax
29,SPB BROK,USD,2018-09-20,0.75,MICROSOFT,div
25,SPB BROK,USD,2018-03-16,0.76,MICROSOFT,div
3,MOEX BROK,RUB,2017-08-01,713.32,ФСК ЕЭС,div
15,MOEX IIS,RUB,2018-06-15,-63.00,ММК,tax
16,MOEX IIS,RUB,2018-06-15,483.60,ММК,div


### 7. Слияние двух датафреймов и получение результата

In [207]:
df_result = pd.merge(df_deals, df_stocks_avg, how='outer', on=['stock_name', 'account', 'currency']).fillna(0)
df_result.sample(10)

,account,brokerage,currency,date_oper,price,quantity,result,stock_name,type_oper,current_cost,current_cost_rub,saldo,avg_open_price
2,MOEX BROK,24.81,RUB,2017-02-03,0.2481,50000.0,-24.810000,ФСК ЕЭС,deal,0.16745,8372.50,50000.0,0.245986
21,MOEX IIS,8.74,RUB,2017-11-22,102.2200,15.0,-8.740000,ОФЗ 24019,deal,100.48900,290987.20,280.0,102.115960
7,MOEX IIS,7.67,RUB,2017-01-03,33.6550,400.0,-7.670000,ММК,deal,51.69500,31017.00,600.0,36.169643
1,MOEX BROK,19.94,RUB,2017-02-03,0.2492,-40000.0,1707.880000,ФСК ЕЭС,deal,0.16745,8372.50,50000.0,0.245986
31,SPB BROK,0.05,USD,2017-09-21,156.0000,1.0,-0.050000,APPLE,deal,225.35000,118449.47,8.0,148.026250
24,MOEX IIS,11.69,RUB,2017-12-28,102.5320,20.0,-11.690000,ОФЗ 24019,deal,100.48900,290987.20,280.0,102.115960
16,MOEX IIS,0.58,RUB,2017-06-23,102.4270,1.0,-0.580000,ОФЗ 24019,deal,100.48900,290987.20,280.0,102.115960
10,MOEX IIS,4.69,RUB,2017-02-07,41.1000,200.0,-4.690000,ММК,deal,51.69500,31017.00,600.0,36.169643
5,MOEX BROK,8.37,RUB,2017-02-27,0.2092,-20000.0,-3420.843571,ФСК ЕЭС,deal,0.16745,8372.50,50000.0,0.245986
18,MOEX IIS,2.92,RUB,2017-07-31,102.4990,5.0,-2.920000,ОФЗ 24019,deal,100.48900,290987.20,280.0,102.115960


### 8. Финальное слияние,  третьего датафрейма с предыдущими

In [208]:
df_result_full = df_result.append(df_flows, ignore_index=True).fillna(0)
df_result_full.sample(10).head()

,account,avg_open_price,brokerage,currency,current_cost,current_cost_rub,date_oper,price,quantity,result,saldo,stock_name,type_oper
36,MOEX BROK,0.000000,0.00,RUB,0.00000,0.0,2017-08-01,0.0000,0.0,-7.00,0.0,ФСК ЕЭС,tax
50,MOEX IIS,0.000000,0.00,RUB,0.00000,0.0,2018-06-15,0.0000,0.0,-63.00,0.0,ММК,tax
49,MOEX IIS,0.000000,0.00,RUB,0.00000,0.0,2018-04-18,0.0000,0.0,12745.60,0.0,ОФЗ 24019,NKD
59,SPB BROK,0.000000,0.00,USD,0.00000,0.0,2018-03-01,0.0000,0.0,4.54,0.0,APPLE,div
1,MOEX BROK,0.245986,19.94,RUB,0.16745,8372.5,2017-02-03,0.2492,-40000.0,1707.88,50000.0,ФСК ЕЭС,deal


### 9. Сохранение результирующего набора в csv

In [209]:
if not exists('OUTPUT'): makedirs('OUTPUT')
report_name = 'OUTPUT\my_trader_diary.csv'

df_result_full.to_csv(report_name, index = False, encoding='utf-8-sig')